# Open Source Asset Liability 

Open Source Asset Liability (OSAL) is a free modelling solution that allows financial companies to simulate their financial performance in the future.

OSAL consists of different modules.

## Modelling parameters

OSM can be configured to the varied needs of the user. The user can specify the length of the modelling (in years), the type of run, the type of curve used as the risk free rate, the country and the specific modelling date. Bellow we describe in more details each specific setting

#### Length of the modelling horizon

ToDo

#### Run type

ToDo

### Selection of the risk free curve

ToDo

### Country

ToDo

#### Modelling date

 - $MD \dots $ modelling date

### Government bond input and calibration

### Corporate bond input and calibration

One of the types of assets that OSAL can support is corporate debt in the form of corporate bonds. Bonds, described in the input files, are calibrated to each specific run

 - $f \dots $ frequency of dates 
 - $M \dots $ maturity date
 - $I \dots $ issue date
 - $cy \dots $ coupon yield
 - $N \dots $ notional amount
 - $MV \dots $ market value at modelling date
 - $BV \dots $ book value at modelling date
  

### 1) Dates

The basic building block of OSAL is a cash flow. The first step in the bond input process is to calculate dates at which the bond is expected to produce a cash flow.

We will denote these dates as:

$$  I\leq t_1 < \dots < t_n \leq M $$

### 2) Refactoring

The first step in refactoring for fixed income assets such as corporate bonds consists of calculating the cash flow dates that occur after the modelling date since the past cash flows are not considered in the modell. For each cash flow date, a time fraction is calculated with regards to the modelling date in years. This facilitates discounting in later stages.



The calculation of the date fractions is as follows:

$ t_1^f = t_j - MD$  where $ j = \min_i(t_i \geq MD)  $

$ t_2^f = t_{j+1} - MD $

$\dots$

$t_n^f = t_{j+k} - MD $


The date fractions are saved and can be viewed in the following object property

dates = $ \{ t_1^f, t_2^f, \dots t_{j+k}^2 \}$

### 3) Cashflow simulation

This section simulates the size of the cash flows in the local currency. For bonds there are two types of cash flows. The coupons and the notional payout.

##### Coupon cash flow

$ divcf = \{ d_1, \dots, d_k \} $

Where:

$ d_i = N*cy  $

##### Notional cash flow


$ notcf = \{ N \} $

Where:

$ d_1 = N $

### 4) Calibration

The calibration is the process of finding the correct discount rate for each bond. The correct discount rate at which the discounted price of the bond is equal to the market value of the bond observed at the modelling date.

Note that if the run is set to Risk Neutral, the sector specific spread is equal to 0 for all bonds.

The discount rate is calibrated using a constant factor called z-spread that increases-decreases the risk free discount rate such that: 

$$
MV = \frac{d_1}{(1+ y_1+ s + z)^{t_1^f}} + \dots+\frac{d_k}{(1+ y_k+ s + z)^{t_k^f}} +\frac{N}{(1+ y_k+ s + z)^{t_k^f}} 
$$

Where
 - $MV\dots $
 -  $ d_i\dots $ is the i-th coupon
 - $ N\dots $ is the notional amount
 - $ y_i \dots $ is the annual yield for a period between the modelling date and time $t_i$
 - $ s \dots  $ is the sector specific spread
 - $ z \dots $ is the z-Spread of the individual bond  

 ## Equity input and calibration

OSAL supports the asset class of equities. Different kinds of shares in the company. The shares are modeled as instruments that offer a constant cash flow in the form of a dividend based on the market value at the time of payment and a constant growth as the company matures. At the end of the modelling period, a terminal value is calculated. The terminal value represents the discounted value of all future cash flows outside of the modelling period.

Each equity asset has the following fields:

 - $f \dots $ frequency of dividend payments 
 - $END \dots $ end of the modelling period
 - $I \dots $ issue date
 - $dy \dots $ dividend yield
 - $MV \dots $ market value at modelling date
 - $BV \dots $ book value at modelling date

### 1) Dates

The first step is to calculate the dates at which the equity asset generates a cash flow since its issue. This can be either a dividend or a terminal value sale.

#### Dividend cash flow dates

$$  I\leq t_1 < \dots < t_n \leq END $$


#### Terminal value date

$t_{term} = END$

### 2) Refactoring

Similar to refactoring for fixed income assets, this step consists of calculating the cash flow dates that occur after the modelling date since the past cash flows are not considered in the modell. For each cash flow date, a time fraction is calculated with regards to the modelling date in years. This facilitates discounting in later stages.

The calculation of the date fraction is:

$ t_1^f = t_j - MD$ 

Where 

$ j = \min_i(t_i \geq MD)  $

$ t_2^f = t_{j+1} - MD $

$\dots$

$t_n^f = t_{j+k} - MD $

The date fractions are saved and can be viewed in the following object property:

dates = $ \{ t_1^f, t_2^f, \dots t_{j+k}^2 \}$

### 3) Calibration

The calibration of an equity asset uses a Gordon growth model to calculate the implied future growth rate, would imply the observed market value at the modelling date. 

The calibration consists of finding a growth rate g such that:

$$
MV = \frac{MV*(1+g)^{t^f_1} dy}{(1+y_1)^{t^f_1}}+\frac{MV*(1+g)^{t^f_2} dy}{(1+y_2)^{t^f_2}}+\dots + \frac{MV*(1+g)^{t^f_m} dy}{(1+y_m)^{t^f_m}} + \frac{1}{(1+y_m)^{t^f_m}} \frac{MV*dy}{r-g}
$$

$$
1 = \frac{(1+g)^{t^f_1} dy}{(1+y_1)^{t^f_1}}+\frac{(1+g)^{t^f_2} dy}{(1+y_2)^{t^f_2}}+\dots + \frac{(1+g)^{t^f_m} dy}{(1+y_m)^{t^f_m}} + \frac{1}{(1+y_m)^{t^f_m}} \frac{dy}{r-g}
$$

$$
\frac{1}{dy} = \frac{(1+g)^{t^f_1}}{(1+y_1)^{t^f_1}}+\frac{(1+g)^{t^f_2}}{(1+y_2)^{t^f_2}}+\dots + \frac{(1+g)^{t^f_m}}{(1+y_m)^{t^f_m}} + \frac{1}{(1+y_m)^{t^f_m}} \frac{1}{r-g}
$$



### 4) Cashflows

After obtaining the growth rate, the growth of the value of the share and by consequence the size of each dividend can be projected (As well as the terminal value). The calculation of the market value is done using the date fractions calculated in step 1, the calculated yields for each period and the growth rate. 

For each time period where a dividend is supposed to be paid, the market value is calculated as:

$MV_{t_i^f} = MV (1+g)^{t^f_i}$

Based on the market value at that period, the cash flow related to the dividend is calculated as:

$d_i = MV_{t_i^f}* dy$

The terminal value of the equity is calculated as

$TV = \frac{MV}{r-g}$


$  divcf =  \{ d_1, \dots, d_k \} $

## Property input and calibration

ToDo

## Cash input

ToDo

# Generation of future investment assets

Companies are expected to generate excess cash from operation, capital return or investment in the future. That cash is normaly invested into the investments available at that time. Most of the assets that will be in the investment universe at that time do not exist yet at the Modelling Date and are therefor not present in the portfolio.

To keep the model realistic, OSAL needs a way to create future assets.

ToDo

## Liability preparation

### Term life

ToDo

### Health

ToDo

### P&C

ToDo

# Projection

ToDo

# Trading

ToDo